LSTM v2.6 (seperate attacks, simplify normalization)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import time

# parameters
num_features = 10
num_scenarios = 12
scenario_length = 1000
attack_type = 0 		# 0: all, 1: DoS, 2: FDI, 3: Replay

# hyper-parameter
seq_len = 20
seq_overlap = seq_len - 1
lstm_blocks = 32
epoch_val = 20
batch_size_val = 4

# 1 data preprocessing
data = pd.read_csv('dataset.csv')

# 1.1 normalization
labels = data['label']
data = data.drop(columns=['time', 'label'])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized['label'] = labels

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

if attack_type == 0:
    scenario_range = range(0, 12)
elif attack_type == 1:
    scenario_range = range(0, 4)
elif attack_type == 2:
    scenario_range = range(4, 8)
elif attack_type == 3:
    scenario_range = range(8, 12)

for s in scenario_range:
		scenario_start = s * scenario_length
		for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
				sequence = data_normalized[i:i + seq_len]
				sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# 3. model creation
# 3.1 model archiecture
model = Sequential()
model.add(Input(shape=(seq_len, num_features)))
model.add(LSTM(lstm_blocks))
model.add(Dense(1, activation='sigmoid'))

# 3.2 model compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# 3.3 model train
start_time = time.time()
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)
end_time = time.time()

# 3.4 model test
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# 3.5 print results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-score: {f1:.3f}')

# 3.6 print training time
training_time = end_time - start_time
print(f"Training Time: {training_time:.1f}")


LSTM v2.7 (seperate scenario for train & test!)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import time

# parameters
num_features = 10
num_scenarios = 12
scenario_length = 1000
attack_type = 0 		# 0: all, 1: DoS, 2: FDI, 3: Replay

# hyper-parameter
seq_len = 20
seq_overlap = seq_len - 1
lstm_blocks = 32
epoch_val = 20
batch_size_val = 4

# 1 data preprocessing
data = pd.read_csv('dataset.csv')

# 1.1 normalization
labels = data['label']
data = data.drop(columns=['time', 'label'])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized['label'] = labels

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

if attack_type == 0:
    scenario_range = range(0, 12)
elif attack_type == 1:
    scenario_range = range(0, 4)
elif attack_type == 2:
    scenario_range = range(4, 8)
elif attack_type == 3:
    scenario_range = range(8, 12)

# train data
for s in [0, 1, 2, 4, 5, 6, 8, 9, 10]:
		scenario_start = s * scenario_length
		for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
				sequence = data_normalized[i:i + seq_len]
				sequences.append(sequence)
X_train = array(sequences)

sequences = []

# test data
for s in [3, 7, 11]:
		scenario_start = s * scenario_length
		for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
				sequence = data_normalized[i:i + seq_len]
				sequences.append(sequence)
X_test = array(sequences)


# 2.2 reshape data for LSTM network
y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# 3. model creation
# 3.1 model archiecture
model = Sequential()
model.add(Input(shape=(seq_len, num_features)))
model.add(LSTM(lstm_blocks))
model.add(Dense(1, activation='sigmoid'))

# 3.2 model compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# 3.3 model train
start_time = time.time()
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)
end_time = time.time()

# 3.4 model test
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# 3.5 print results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-score: {f1:.3f}')

# 3.6 print training time
training_time = end_time - start_time
print(f"Training Time: {training_time:.1f}")


LSTM v2.5 (add preprocessing, code organizing)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# parameters
num_features = 10
num_scenarios = 12
scenario_length = 1000

# hyper-parameter
seq_len = 1
seq_overlap = 0
lstm_blocks = 64
epoch_val = 20
batch_size_val = 4
data_normalization = 1

# 1. data preprocessing
data = pd.read_csv('dataset.csv')

# 1.1 normalization
if data_normalization == 0:
	data = data.drop(columns=['time'])
	data_normalized = data
else:
	labels = data['label']
	data = data.drop(columns=['time', 'label'])

	if data_normalization == 1:
		scaler = StandardScaler()
	elif data_normalization == 2:
		scaler = MinMaxScaler()

	data_normalized = scaler.fit_transform(data)
	data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
	data_normalized['label'] = labels

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap
for s in range(num_scenarios):
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# 3. model creation
# 3.1 model archiecture
model = Sequential()
model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
model.add(Dense(1, activation='sigmoid'))

# 3.2 model compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# 3.3 model train
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)

# 3.4 model test
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# 3.5 print results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-score: {f1:.3f}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                19200     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 19265 (75.25 KB)
Trainable params: 19265 (75.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20
2248/2248 [==============================] - 10s 4ms/step - loss: 0.4380 - accuracy: 0.8435
Epoch 2/20
2248/2248 [==============================] - 10s 5ms/step - loss: 0.4152 - accuracy: 0.8468
Epoch 3/20
2248/2248 [==============================] - 6s 3ms/step - loss: 0.4085 - accuracy: 0.8466
Epoch 4/20
2248/2248 [==============================] - 8s 4ms/step - loss: 0.4012 - accuracy: 0.8466
E

LSTM v2.4 (add 1d-CNN) (not working)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

num_features = 10

seq_len = 40
lstm_blocks = 20
epoch_val = 20
batch_size_val = 4

# load
train_data = pd.read_csv('dataset_train.csv')
test_data = pd.read_csv('dataset_test.csv')

# drop time
train_data = train_data.drop(columns=['time'])
test_data = test_data.drop(columns=['time'])

# split into samples
samples = list()

for i in range(0,1000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)
for i in range(1000,2000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)
for i in range(2000,3000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)

for i in range(0,1000-seq_len):
	sample = test_data[i:i+seq_len]
	samples.append(sample)


data = array(samples)
data_reshaped = data.reshape(data.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data.shape[1], data.shape[2])
X_test = X_test.reshape(X_test.shape[0], data.shape[1], data.shape[2])

y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# Build the LSTM model
model = Sequential()
# model.add(Conv1D(filters=8, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

LSTM v2.3 (add dropout)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

num_features = 10

seq_len = 40
lstm_blocks = 20
epoch_val = 20
batch_size_val = 4

dropout_ver = 2

# load
train_data = pd.read_csv('dataset_train.csv')
test_data = pd.read_csv('dataset_test.csv')

# drop time
train_data = train_data.drop(columns=['time'])
test_data = test_data.drop(columns=['time'])

# split into samples
samples = list()

for i in range(0,1000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)
for i in range(1000,2000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)
for i in range(2000,3000-seq_len):
	sample = train_data[i:i+seq_len]
	samples.append(sample)

for i in range(0,1000-seq_len):
	sample = test_data[i:i+seq_len]
	samples.append(sample)


data = array(samples)
data_reshaped = data.reshape(data.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data.shape[1], data.shape[2])
X_test = X_test.reshape(X_test.shape[0], data.shape[1], data.shape[2])

y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# Build the LSTM model
if dropout_ver == 1:
	model = Sequential()
	model.add(LSTM(lstm_blocks, dropout=0.2, recurrent_dropout=0.2, input_shape=(seq_len, num_features)))
	model.add(Dense(1, activation='sigmoid'))
elif dropout_ver == 2:
	model = Sequential()
	model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
	model.add(Dropout(0.2))
	model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

LSTM v2.2 (split all data into train and test randomly)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

num_features = 10

seq_len = 40
lstm_blocks = 20
epoch_val = 20
batch_size_val = 4

# load
data = pd.read_csv('dataset.csv')

# drop time
data = data.drop(columns=['time'])

# split into samples
samples = list()

for i in range(0,1000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(1000,2000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(2000,3000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(0,3000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)


data = array(samples)
data_reshaped = data.reshape(data.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data.shape[1], data.shape[2])
X_test = X_test.reshape(X_test.shape[0], data.shape[1], data.shape[2])

y_train = X_train[:, -1, -1]
y_test = X_test[:, -1, -1]
X_train = X_train[:, :, :-1]
X_test = X_test[:, :, :-1]

# Build the LSTM model
model = Sequential()
model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=epoch_val, batch_size=batch_size_val)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20)                2480      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2501 (9.77 KB)
Trainable params: 2501 (9.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20
1095/1095 [==============================] - 14s 11ms/step - loss: 0.4892 - accuracy: 0.7950
Epoch 2/20
1095/1095 [==============================] - 12s 11ms/step - loss: 0.4399 - accuracy: 0.8023
Epoch 3/20
 159/1095 [===>..........................] - ETA: 9s - loss: 0.3918 - accuracy: 0.8082

LSTM v2.1 (change dataset files)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# n = 4000
num_features = 10;

seq_len = 5
lstm_blocks = 50;
epoch = 20;
batch_sizee = 4;

# load
train_data = pd.read_csv('dataset_train.csv')
test_data = pd.read_csv('dataset_test.csv')

# drop time
train_data = train_data.drop(columns=['time'])
test_data = test_data.drop(columns=['time'])

# split into samples
samples_train = list()
samples_test = list()

for i in range(0,1000-seq_len):
	sample = train_data[i:i+seq_len]
	samples_train.append(sample)
for i in range(1000,2000-seq_len):
	sample = train_data[i:i+seq_len]
	samples_train.append(sample)
for i in range(2000,3000-seq_len):
	sample = train_data[i:i+seq_len]
	samples_train.append(sample)

for i in range(0,1000-seq_len):
	sample = test_data[i:i+seq_len]
	samples_test.append(sample)

# reshape subsequences
train_data = array(samples_train)
test_data = array(samples_test)

X_train = train_data[:, :, :num_features]
X_test = test_data[:, :, :num_features]
y_train = train_data[:, -1, -1]
y_test = test_data[:, -1, -1]

# Build the LSTM model
model = Sequential()
model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=epoch, batch_size=batch_sizee)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
# print(f'F1-score: {f1}')

LSTM v2 (appropraite sequence generation)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

n = 4000
num_features = 10;

seq_len = 5
split_index = 3400;
lstm_blocks = 50;
epoch = 20;
batch_sizee = 4;

# load
data = pd.read_csv('dataset.csv')
# drop time
data = data.drop(columns=['time'])

# split into samples
samples = list()
for i in range(0,1000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(1000,2000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(2000,3000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)
for i in range(3000,4000-seq_len):
	sample = data[i:i+seq_len]
	samples.append(sample)

# reshape subsequences
data = array(samples)

X_train = data[:split_index, :, :num_features]
X_test = data[split_index:, :, :num_features]
y_train = data[:split_index, -1, -1]
y_test = data[split_index:, -1, -1]

# Build the LSTM model
model = Sequential()
model.add(LSTM(lstm_blocks, input_shape=(seq_len, num_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=epoch, batch_size=batch_sizee)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
# print(f'F1-score: {f1}')



X_train shape: (3000, 5, 10)
X_test shape: (980, 5, 10)
y_train shape: (3000,)
y_test shape: (980,)

LSTM v1 (basic)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the dataset
data = pd.read_csv('dataset.csv')
num_data, num_features = data.shape
num_features = num_features - 1
split_index = int(num_data * 0.8)

# Split the data into training and test sets
X_train = data.iloc[:split_index, :-1]
y_train = data.iloc[:split_index, -1]
X_test = data.iloc[split_index:, :-1]
y_test = data.iloc[split_index:, -1]

# Build the LSTM model
model = Sequential()
model.add(LSTM(20, input_shape=(1, num_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = X_train.to_numpy().reshape(32, 1, num_features)
y_train = y_train.to_numpy().reshape(32, 1, 1)
X_test = X_test.to_numpy().reshape(12, 1, num_features)
y_test = y_test.to_numpy().reshape(12, 1, 1)

# Train the LSTM model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')